# Analysis of the Age of Grand Slam Tennis Finalists
An excercise in manipulation of pandas dataframes and data visualization  

## 1) Data Scraping 
N.B. These are already in csv files that I copied-and-pasted from the sites given below (see commented bit at the start of the Data Manipulation section). This is just an exercise

Start with the players list and birth years from Wikipedia

In [1]:
import requests
from bs4 import BeautifulSoup
from unidecode import unidecode
import lxml
import ipdb
import numpy as np
import pandas as pd


req = requests.get('https://en.wikipedia.org/wiki/List_of_male_singles_tennis_players')
soup = BeautifulSoup(req.content, 'lxml')
table_classes = {"class": ["sortable", "plainrowheaders"]}
tennisPlayers = soup.findAll("table", table_classes)

playerNames = []
birthYears = []
for row in tennisPlayers[0].find_all("tr"):
    countCol = 0
    for item in row.find_all("td"):
        if countCol == 0:
            #Only Pick the Name not the Country and remove accents from names for ease of comparison
            name = unidecode(row.find_all("a")[0]['title'])
            playerNames.append(name)
        elif countCol == 1:
            #Get year remove the <td> bits
            birthYears.append(str(item)[4:-5])
        countCol = countCol + 1

List of Grandslam winners and runners-up from ESPN

In [2]:
req = requests.get('http://www.espn.com/tennis/history')
soup = BeautifulSoup(req.content, 'lxml')
table_classes = {"class": ["sortable", "plainrowheaders","tablehead"]}
gsWinners = soup.findAll("table", table_classes)


winners = []
rus = []
year = []
tournament = []
countRow = 0
for row in gsWinners[0].find_all("tr"):
     countCol = 0
     if countRow > 1:
        for item in row.find_all("td"):
            if countCol == 0:
                year.append(str(item)[4:-5])
            elif countCol == 1:
                tournament.append(str(item)[4:-5])
            elif countCol == 2:
                name = str(item.find_all("a"))
                #Format Change at 2006, with no Link just text
                if name == '[]':
                    winners.append(unidecode(str(item)[4:-5]))
                else:
                    index = name.find("\">")
                    winners.append(unidecode(name[index+2:-5]))
            elif countCol == 3:
                name = str(item.find_all("a"))
                if name == '[]':
                    rus.append(unidecode(str(item)[4:-5]))
                else:
                    index = name.find("\">")
                    rus.append(unidecode(name[index + 2:-5]))
            countCol = countCol + 1
     countRow = countRow + 1

# 2) Data Cleaning 
Some of these entries are going to be difficult to concatenate between the two as they end in stuff like (tennis) or their name ends in Jr. or Sr. So remove all these

In [3]:
nameStringsToCheck = [playerNames, winners, rus]

extraStringsToRemove = [" (tennis)",
                        " (page does not exist)",
                        " (figure skater)",
                        ", Jr.",
                        " Jr.",
                        ", Sr.",
                        ", III",
                        ", 2d",]

for listOfNames in nameStringsToCheck:
    for stringToCheck in extraStringsToRemove:
        for i in range(len(listOfNames)):
            if listOfNames[i].find(stringToCheck) != -1:
                listOfNames[i] = listOfNames[i][:listOfNames[i].find(stringToCheck)]

#One birth year says c.1921 which doesn't convert to float well
for i in range(len(birthYears)):
    if birthYears[i].find("c.") != -1:
        birthYears[i] = birthYears[i][birthYears[i].find("c.")+2:-1]
    if birthYears[i].find("?") != -1:
        birthYears[i] = "NaN"                
                
#Note that the capitalization is to make sure consistent with csv files                 
playersList = pd.DataFrame(
    {'Name': playerNames,
     'Birth': birthYears,
    })

gsWinners = pd.DataFrame(
     {'YEAR': year,
      'TOURNAMENT': tournament,
      'WINNER': winners,
      'RUNNER-UP': rus,
     })

## 3) Data Manipulation
Get Data into a form that can be used for plotting

In [4]:
import matplotlib.pyplot as plt
import pandas as pd
import datetime
import math
import numpy as np

#If you don't want to do the data scraping and cleaning uncomment the lines
#below and start from here

gsWinners = pd.read_csv('GrandSlamWinners2.csv', encoding='latin1')
playersList = pd.read_csv('TennisPlayers.csv', encoding='latin')

gsWinners['YEAR'] = gsWinners['YEAR'].astype('int64')
playersList['Birth'] = playersList['Birth'].astype('int64')

#Change Name to First 2 Letters of First Name Plus Surname To Compare acorss dataFrames
playersList['NameShortened'] = playersList['Name'].map(lambda string: string[0] + string[string.rfind(' ')+1:])
gsWinners['WinnerShortened'] = gsWinners['WINNER'].map(lambda string: string[0] + string[string.rfind(' ')+1:])
gsWinners['RUShortened'] = gsWinners['RUNNER-UP'].map(lambda string: string[0] + string[string.rfind(' ')+1:])

#Compare DataFrame Containing Names and BirthYears with That of Winners and Runners-Up
playersList.set_index('NameShortened',inplace=True)
gsWinners.set_index('WinnerShortened',inplace=True)
#Add a column for winner Birth Year
gsWinners = gsWinners.join(playersList).rename(columns={'Birth':'WinnerBirthYear','Name':'WinnerNameCheck'})
#Do same for Runner up
gsWinners.set_index('RUShortened',inplace=True)
gsWinners = gsWinners.join(playersList).rename(columns={'Birth':'RUBirthYear','Name':'RUNameCheck'})
gsWinners.reset_index(inplace=True)

#Calculate Ages
gsWinners['WinnerAge'] = gsWinners['YEAR'] - gsWinners['WinnerBirthYear']
gsWinners['RUAge'] = gsWinners['YEAR'] - gsWinners['RUBirthYear']

#Mix-up between Sam Smith and Sidney Smith who were born 74 years apart. 
gsWinners['WinnerAge'] = gsWinners['WinnerAge'].map(lambda x: np.nan if (x > 50 or x<15)else x)
gsWinners['RUAge'] = gsWinners['RUAge'].map(lambda x: np.nan if (x > 50 or x <15) else x)

#Get year plus month as decimal for plotting
#Dictionary to change year and month based on a fraction
fractionalDates = {'Wimbledon':0.6,
                   'U.S. Open': 0.85,
                   'French Open': 0.35,
                   'Australian Open': 0.1,
                   'Australian Open (Jan)': 0.1,
                   'Australian Open (Dec)': 0.99}


gsWinners['YearExact'] = gsWinners['YEAR'] + gsWinners['TOURNAMENT'].replace(fractionalDates).astype('float32')
gsWinners = gsWinners.sort_values('YearExact')

#Smooth out variations with an 8 Slam Rolling Mean
gsWinners['Winner (8 Slam Rolling Mean)'] = gsWinners['WinnerAge'].rolling(window = 8, min_periods = 1,center=False).mean()
gsWinners['Runner-Up (8 Slam Rolling Mean)'] = gsWinners['RUAge'].rolling(window = 8, min_periods = 1,center=False).mean()        


# 4) Plotting

In [5]:
from bokeh.plotting import figure, output_file, ColumnDataSource
from bokeh.models import HoverTool
from bokeh.io import output_notebook,show


scatterColours = {'Wimbledon':'green',
                  'U.S. Open':'black',
                  'French Open': 'orange',
                  'Australian Open':'blue', 
                  'Australian Open (Jan)': 'blue', 
                  'Australian Open (Dec)': 'blue'}

gsWinners['scatterColours'] = gsWinners['TOURNAMENT'].replace(scatterColours) 

#Drop NaNs Otherwise tooltips don't line up with scatter
gsWinners.dropna(subset=['WinnerAge'],inplace=True)

gsWinners['ToolTip'] = gsWinners['TOURNAMENT'] + ' ' \
                       + gsWinners['YEAR'].astype(str) + ': ' \
                       + gsWinners['WINNER'] + ' beat ' \
                       + gsWinners['RUNNER-UP']


#Scatter Plot shows ages of winners. Line plots for ages of winners and runners-up
p = figure(plot_height = 600, plot_width = 750,
           title = 'Age of Grandslam Tennis Finalists',
           x_axis_label = 'Year',
           y_axis_label = 'Age',
           x_range=(1970, 2020),
           y_range=(15, 45)
           )

gsWinnerColumnSource = ColumnDataSource(gsWinners)

p.circle(source = gsWinnerColumnSource,
         x = 'YearExact',
         y = 'WinnerAge',
         color = 'scatterColours',
         alpha = 0.2,
         name = 'scatter')

p.line(source = gsWinnerColumnSource,
       x = 'YearExact',
        y = 'Winner (8 Slam Rolling Mean)',
         color = 'navy',
         line_width=1.5,
         legend = 'Winner (8 Slam Mean)')

p.line(source = gsWinnerColumnSource,
         x = 'YearExact',
         y = 'Runner-Up (8 Slam Rolling Mean)',
         color = 'orange',
        line_width=1.5,
        legend='Runner-Up (8 Slam Mean)')

p.legend.location = "top_left"
p.legend.click_policy="hide"
toolTip = HoverTool(tooltips = '@ToolTip',names = ['scatter'])
p.add_tools(toolTip)

#output_file("grandSlamWinners.html")
output_notebook()
show(p)

Loading BokehJS ...

# Does the older or younger player win more often?

In [6]:
gsWinners['AgeDifference'] = gsWinners['WinnerAge'] - gsWinners['RUAge']
gsWinners.dropna(subset=['AgeDifference'],inplace=True)
#Count Number of times older or younger player wins
isOlder = gsWinners['AgeDifference'].astype(int).map(lambda x: np.sign(x)).replace({0:np.nan}).dropna().value_counts()
print("Fraction of grandslam finals won by the older player:")
print(round(isOlder.loc[1]/float(isOlder.sum()),2))

Fraction of grandslam finals won by the older player:
0.48
